# Mini Detection
This mini detection algorithm allows you to visually fine-tune mini event detection parameters for each recording. 

**What you need:** 
1. Each recording consists of one folder with all of the sweeps (.mat files)
2. For blind detection, each recording's condition should be recorded on the metadata csv (see Github for template)

**What you'll get:**
1. A JSON file of all the parameters you used for each cell for future reference
2. A pickle file of all results for future reference
3. The avg mini Hz and amplitude per recording (sorted by condition)
4. The IPIs for each cell (sorted by condition)
5. Rs, Rm, and Cm values calculated from each sweep's RC check

# Parameter Tuning

In [ ]:
# Import tuner
import param_tune
from param_tune import ParameterTuner

# Specify folder with data and the mode (EPSC or IPSC)
parent_folder = ''  # Update this path

# Load the tuner
tuner = ParameterTuner(parent_folder, mode='EPSC', control_pulse_timing='early')

# Define parameter sets to test
# Format: (window_size, sample_step, interval)
parameter_sets = [
    (130, 35, 20), 
    # (200, 50, 70),  
    (115, 35, 20),   
    (120, 30, 10),
    # (300, 50, 10),
    # (175, 30, 20), 
    # (175, 35, 20)
    # (300, 50, 70),
    (60, 15, 5)
    # (300, 50, 70)
]

# Test parameters on a specific cell
cell_name = ''  # Replace with actual cell name

results = tuner.sample_and_test(cell_name, parameter_sets, n_files=3, visualize=True)

In [ ]:
# Save optimal parameters for cells after testing
# Note: Now you can specify control_pulse_timing='early' or 'late' for each cell individually

tuner.save_parameters("cellname", window_size=115, sample_step=35, interval=20, control_pulse_timing='early')

# Specify output dir
output_dir = ''

# Export parameters to JSON file for later use
tuner.export_parameters("20251129_optimized_parameters.json", output_dir)

# Detection

In [ ]:
# Run full detection with optimized parameters
optimized_results = tuner.run_full_detection(output_dir = output_dir)

# Save optimized_results to disk (pickle)
import os
from io_helpers import save_optimized_results

save_path_pkl = os.path.join(output_dir, "optimized_results.pkl")
save_optimized_results(save_path_pkl, optimized_results, use_pickle=True)

# Group & Export Data

In [ ]:
# Import metadata file
import pandas as pd

# Load metadata csv
metadata = pd.read_csv('')

# Examine metadata structure
print("Metadata columns:", metadata.columns.tolist())
print("\nFirst few rows of metadata:")
print(metadata.head())
print("\nUnique values in each column:")
for col in metadata.columns:
    print(f"{col}: {metadata[col].unique()}")
print(f"\nMetadata shape: {metadata.shape}")

In [ ]:
import functions

# Separate data by groups defined by metadata
grouped_data, ungrouped = functions.separate_data_by_groups(optimized_results, metadata, 
                                                 cell_id_col='Cell', 
                                                 group_col='Condition')

In [ ]:
# Export grouped data to separate CSV files

from functions import export_data, avg_amp_per_cell, hz_per_cell

# Specify date
date = ''

# Ensure output_dir has trailing slash
if not output_dir.endswith('/'):
    output_dir = output_dir + '/'

# Prepare data for export
condition_labels = []
amplitude_data = []
frequency_data = []
detection_data_list = []

for group_name, group_results in grouped_data.items():
    condition_labels.append(group_name)
    
    # Calculate amplitudes and frequencies for this group
    group_amplitudes = avg_amp_per_cell(group_results)
    group_frequencies = hz_per_cell(group_results)
    
    amplitude_data.append(group_amplitudes)
    frequency_data.append(group_frequencies)
    detection_data_list.append(group_results)

# Export amplitudes
export_data(
    condition_labels=condition_labels,
    big_data=amplitude_data,
    data_type='Amplitude',
    date=date,
    output_dir=output_dir,
    detected_data_list=detection_data_list
)

# Export frequencies
export_data(
    condition_labels=condition_labels,
    big_data=frequency_data,
    data_type='Frequency',
    date=date,
    output_dir=output_dir,
    detected_data_list=detection_data_list
)

print("\n=== Export Summary ===")
print(f"Exported data for {len(condition_labels)} groups:")
for i, label in enumerate(condition_labels):
    print(f"  {label}: {len(amplitude_data[i])} cells")
print(f"\nFiles saved to: {output_dir}")
print("Created files:")
print("  - Individual group CSV files for amplitudes and frequencies")
print("  - Combined CSV files with all groups together")

# Additional Analyses

In [ ]:
# Extract and export inter-peak interval (IPI) data organized by condition

from functions import export_ipi_by_condition 

# Export with condition-based organization
export_summary = export_ipi_by_condition(
    detected_data_list=optimized_results,
    output_dir=output_dir,
    date='',
    metadata=metadata,
    cell_id_col='Cell',
    group_col='Condition',
    min_interval_ms=1,
    pad_method='nan'  # or 'drop' or 'repeat_last'
)

In [ ]:
# Compute and export Rs (series resistance), Rm (membrane resistance), and Cm (membrane capacitance) data

from functions import export_Rs_Rm_Cm_data

# Load the parameters from JSON to get control_pulse_timing for each cell
import json
json_path = f"{output_dir}/20251129_optimized_parameters.json" # change json file as needed

with open(json_path, 'r') as f:
    parameters = json.load(f)

print(f"Loaded parameters for {len(parameters)} cells from JSON")

# Export Rs, Rm, and Cm data with cell-specific control_pulse_timing
Rs_Rm_Cm_df = export_Rs_Rm_Cm_data(
    detected_data_list=optimized_results,
    output_dir=output_dir,
    date='',
    metadata=metadata,
    cell_id_col='Cell',
    group_col='Condition',
    control_pulse_timing='early',  # Default fallback if not in JSON
    parameters_dict=parameters,  # Contains cell-specific control_pulse_timing
    fs=10000  # Sampling frequency
)

In [ ]:
# Refresh functions module to pick up any changes made to functions.py
import importlib
import functions
import param_tune


importlib.reload(functions)
importlib.reload(param_tune)


print("Modules refreshed successfully!")